In [1]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

### RESNET 44

In [2]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [3]:
# Need to add this to requirements.txt
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [4]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/resnet.yaml \
    model.resnet.depth 44 \
    train.batch_size 128 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_44/exp00

[2020-06-14 18:03:16] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 44
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_typ

In [5]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 44 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_44/exp00/checkpoint_00160.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_44/exp00/test_results_0160

[2020-06-14 20:40:30] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_44/exp00/checkpoint_00160.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 20.01it/s]
[2020-06-14 20:40:35] __main__ INFO: Elapsed 3.95
[2020-06-14 20:40:35] __main__ INFO: Loss 0.4046 Accuracy 0.9209


In [6]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Right the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnet.yaml \
   model.resnet.depth 44 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_44/exp00/checkpoint_00100.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_44/exp00/test_results_0100

[2020-06-14 20:41:13] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_44/exp00/checkpoint_00100.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:03<00:00, 20.06it/s]
[2020-06-14 20:41:18] __main__ INFO: Elapsed 3.94
[2020-06-14 20:41:18] __main__ INFO: Loss 0.3648 Accuracy 0.9178


In [7]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Right the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/resnet.yaml \
    model.resnet.depth 44 \
    test.batch_size 128 \
    dataset.name CIFAR101 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/resnet_basic_44/exp00/checkpoint_00160.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/resnet_basic_44/exp00/test_results_0160_CIFAR101

[2020-06-14 20:41:44] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnet_basic_44/exp00/checkpoint_00160.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:01<00:00, 14.30it/s]
[2020-06-14 20:41:45] __main__ INFO: Elapsed 1.12
[2020-06-14 20:41:45] __main__ INFO: Loss 0.8338 Accuracy 0.8290


In [8]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['resnet_basic_44', 'resnet_basic_44', 'resnet_basic_44'],
           'Testset': ['cifar10', 'cifar10', 'cifar10.1'],
           'Epoch': [100, 160, 160],
           'Loss': [0.3648, 0.4046, 0.8338],
           'Accuracy': [0.9178, 0.9209, 0.8290],
           'Original_Accuracy': [93.0, 93.0, 84.2],
           'Original_CI': [(92.5, 93.5), (92.5, 93.5), (82.6, 85.8)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])


df.to_csv('/home/ec2-user/SageMaker/experiments/resnet_basic_44/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnet_basic_44,cifar10,100,0.3648,0.9178,93.0,"(92.5, 93.5)"
1,resnet_basic_44,cifar10,160,0.4046,0.9209,93.0,"(92.5, 93.5)"
2,resnet_basic_44,cifar10.1,160,0.8338,0.8290,84.2,"(82.6, 85.8)"


In [9]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/resnet_basic_44/exp00/test_results_0160/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

['preds', 'probs', 'labels', 'loss', 'acc']


array([[-6.475453  , -0.9687562 , -0.2566425 , ..., -3.5769613 ,
        -4.25939   , -7.433777  ],
       [ 5.978677  ,  9.19418   , -9.904226  , ..., -4.364771  ,
        25.662886  ,  0.427105  ],
       [ 2.840697  ,  8.600068  , -5.266426  , ..., -5.3270216 ,
        21.799158  ,  2.9376364 ],
       ...,
       [-5.302427  , -2.2392938 ,  1.7319372 , ...,  2.899787  ,
        -4.9618964 , -3.5398843 ],
       [-0.9742136 , 20.878643  , -0.58532417, ..., -4.4967275 ,
        -2.1827762 , -6.1017246 ],
       [-7.23939   ,  1.4810394 , -3.2119744 , ..., 24.313166  ,
        -5.5873327 , -1.2864003 ]], dtype=float32)

In [10]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/resnet_basic_44'
path = '/home/ec2-user/SageMaker/experiments/resnet_basic_44'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)